In [101]:
from math import sqrt
import numpy as np
from matplotlib import pyplot as plt
from scipy.stats import norm

In [102]:
import numpy as np
from scipy.stats import norm

S0 = 1.00
K = 1.08
T = 1
num_steps = 100
sigma = 0.2
mu = 0.08
r = 0.04
n_path = 10_000


def gbm(S0, r, sigma, t, n_path):
    drift = r * t
    time_steps = np.diff(t, prepend=0)
    normals = np.random.standard_normal((n_path, num_steps+1))
    normals = np.vstack([normals, -normals])
    vol = sigma * np.sqrt(time_steps) * normals.cumsum(axis=1)
    exponent = drift + vol
    paths = S0 * np.exp(exponent)
    return paths

t = np.linspace(0, T, num_steps + 1)
S = gbm(S0, r, sigma, t, n_path)
S

S= stock.prices
S

array([[1.        , 1.00094342, 1.00481152, ..., 0.92573673, 0.94272987,
        0.95573945],
       [1.        , 0.99054454, 0.95261323, ..., 0.92857777, 0.93592109,
        0.91094436],
       [1.        , 0.98802818, 0.99678191, ..., 0.92807148, 0.93179125,
        0.95220372],
       ...,
       [1.        , 1.06302536, 1.09737097, ..., 1.01857982, 0.99684835,
        0.98772544],
       [1.        , 1.02711552, 1.02871529, ..., 0.6845403 , 0.68537906,
        0.67205168],
       [1.        , 0.97741744, 1.00305939, ..., 1.18037425, 1.1362293 ,
        1.15496392]])

In [103]:
def _d1(S, K, r, sigma, t, T):
    t2m = T - t
    numerator = np.log(S / K) + (r + 0.5 * sigma**2) * t2m
    denominator = sigma * np.sqrt(t2m)
    with np.errstate(divide='ignore'):
        return numerator / denominator

def _d2(d1, sigma, t, T):
    t2m = T - t
    return d1 - sigma * np.sqrt(t2m)

def call(S, K, r, t, T, d1, d2):
    return S * norm.cdf(d1) - K * np.exp(-r * (T - t)) * norm.cdf(d2)

def put(S, K, r, t, T, d1, d2):
    return -S * norm.cdf(-d1) + K * np.exp(-r * (T - t)) * norm.cdf(-d2)

def call_delta(d1):
    return norm.cdf(d1)

def put_delta(d1):
    return norm.cdf(d1) - 1

d1 = _d1(S, K, r, sigma, t, T)
d2 = _d2(d1, sigma, t, T)

c = call(S, K, r, t, T, d1, d2)
p = put(S, K, r, t, T, d1, d2)

In [104]:
print(f'Call={c[0, 0]}')
print(f'Put={p[0, 0]}')

Call=0.06370611880864996
Put=0.10135871309315903


In [105]:
delta = call_delta(d1)[:, :-1]
delta

array([[4.66208127e-001, 4.66723438e-001, 4.73098390e-001, ...,
        4.41285188e-007, 3.20864306e-008, 6.58187672e-012],
       [4.66208127e-001, 4.45914952e-001, 3.68087487e-001, ...,
        1.72741954e-006, 5.84018788e-008, 5.04557744e-013],
       [4.66208127e-001, 4.40867087e-001, 4.56995009e-001, ...,
        2.93041284e-007, 5.25405424e-008, 9.88254392e-014],
       ...,
       [4.66208127e-001, 5.86630876e-001, 6.47127228e-001, ...,
        8.28504444e-002, 2.12959719e-002, 3.50592168e-005],
       [4.66208127e-001, 5.18423938e-001, 5.20442295e-001, ...,
        2.48037255e-040, 1.80492890e-058, 1.90102412e-114],
       [4.66208127e-001, 4.19555728e-001, 4.69590839e-001, ...,
        9.93225480e-001, 9.99274912e-001, 9.94881293e-001]])

In [106]:
discount_factors = np.exp(-r * t)
txns = np.diff(delta, axis=1, prepend=0, append=0) * S
txns[:, -1] += np.maximum(S[:, -1] - K, 0)
disounted_txns = discount_factors * txns
costs = disounted_txns.sum(axis=1)

np.mean(costs)

0.06407922389550762

In [107]:
discount_factors = np.exp(-r * t)
txns = np.diff(delta - 1, axis=1, prepend=0, append=0) * S
txns[:, -1] += np.maximum(K - S[:, -1], 0)
disounted_txns = discount_factors * txns
costs = disounted_txns.sum(axis=1)
np.mean(costs)

0.10173181818001674

In [108]:
from typing import Protocol, Sequence
from dataclasses import dataclass

class Stock:
    def __init__(self, mu: float, r: float, sigma: float, time_steps: Sequence[float], cost: float = 0.0) -> None:
        self.mu = mu
        self.r = r
        self.sigma = sigma
        self.time_steps = time_steps
        self.cost = cost

        self.prices: np.ndarray | None = None

    def simulate(self, num_sims: int) -> None:
        drift = self.mu * self.time_steps
        time_increments = np.diff(self.time_steps, prepend=0)
        normals = np.random.standard_normal((num_sims // 2, len(self.time_steps)))
        normals = np.vstack([normals, -normals])
        vol = self.sigma * np.sqrt(time_increments) * normals.cumsum(axis=1)
        exponent = drift + vol
        self.prices = np.exp(exponent)

    def discount_factors(self) -> None:
        discount_factors = np.exp(-self.r * self.time_steps)
        return discount_factors

@dataclass
class EuropeanOption:
    stock: Stock
    strike: float
    maturity: float
    is_call: bool

class HedgingStrategy(Protocol):

    @property
    def stock(self) -> Stock:
        ...
    
    @property
    def derivative(self) -> EuropeanOption:
        ...
    
    def get_hedge_ratio(self, time_step_idx: int | None = None):
        ...

@dataclass
class BlackScholesHedgingStrategy:

    option: EuropeanOption

    @property
    def stock(self) -> Stock:
        return self.option.stock
    
    @property
    def derivative(self) -> EuropeanOption:
        return self.option
    
    def get_hedge_ratio(self, time_step_idx: int | None = None):
        if time_step_idx:
            raise NotImplementedError('TODO')

        stock = self.option.stock
        S = stock.prices
        r = stock.r
        K = self.option.strike
        d = BlackScholesHedgingStrategy.d1(S, K, stock.r, stock.sigma, stock.time_steps, self.option.maturity)

        if self.option.is_call:
            return BlackScholesHedgingStrategy.call_delta(d)
        
        return BlackScholesHedgingStrategy.put_delta(d)

    @staticmethod
    def d1(S, K, r, sigma, t, T):
        t2m = T - t
        numerator = np.log(S / K) + (r + 0.5 * sigma**2) * t2m
        denominator = sigma * np.sqrt(t2m)

        with np.errstate(divide='ignore'):
            return numerator / denominator

    @staticmethod
    def d2(d1, sigma, t, T):
        t2m = T - t
        return d1 - sigma * np.sqrt(t2m)
    
    @staticmethod
    def call_price(S, K, r, t, T, d1, d2):
        return S * norm.cdf(d1) - K * np.exp(-r * (T - t)) * norm.cdf(d2)
    
    @staticmethod
    def put_price(S, K, r, t, T, d1, d2):
        return -S * norm.cdf(-d1) + K * np.exp(-r * (T - t)) * norm.cdf(-d2)
    
    @staticmethod
    def call_delta(d1):
        return norm.cdf(d1)
    
    @staticmethod
    def put_delta(d1):
        return norm.cdf(d1) - 1

class Pricer:
    def price(self, hedging_strategy: HedgingStrategy) -> float:
        S = hedging_strategy.stock.prices
        discount_factors = hedging_strategy.stock.discount_factors()
        delta = hedging_strategy.get_hedge_ratio()[:, :-1]
        txns = np.diff(delta, axis=1, prepend=0, append=0) * S
        
        payout = np.maximum(S[:, -1] - K, 0)
        txns[:, -1] += payout
        disounted_txns = discount_factors * txns
        costs = disounted_txns.sum(axis=1)
        
        return np.mean(costs)

In [110]:
import numpy as np

K = 1.08
T = 1
num_steps = 100
sigma = 0.2
mu = 0.04
r = 0.04
n_path = 10_000
t = np.linspace(0, T, num_steps + 1)


stock = Stock(mu=r, r=r, sigma=sigma, time_steps=t)
stock.simulate(n_path)
stock.prices

array([[1.        , 1.00410242, 1.01181463, ..., 0.85130459, 0.88608392,
        0.91967041],
       [1.        , 1.06021222, 1.04245734, ..., 1.11158554, 1.14051797,
        1.12999349],
       [1.        , 0.9894019 , 1.02084505, ..., 1.22997816, 1.2356688 ,
        1.25695148],
       ...,
       [1.        , 0.97895449, 1.00779266, ..., 0.86411158, 0.84878838,
        0.8385603 ],
       [1.        , 1.04275904, 1.0655112 , ..., 1.03823363, 1.02895629,
        1.02622797],
       [1.        , 1.03558333, 1.03281781, ..., 1.19628004, 1.21100927,
        1.26932351]])

In [111]:
call_option = EuropeanOption(stock, K, T, is_call=True)
bs_strategy = BlackScholesHedgingStrategy(call_option)
bs_strategy.get_hedge_ratio()[:,:-1]

array([[4.66208127e-01, 4.73022463e-01, 4.87074944e-01, ...,
        9.74336694e-12, 2.87488145e-17, 2.94488993e-23],
       [4.66208127e-01, 5.81436863e-01, 5.47080035e-01, ...,
        7.03375601e-01, 8.55789612e-01, 9.97075012e-01],
       [4.66208127e-01, 4.43623236e-01, 5.04976266e-01, ...,
        9.99954968e-01, 9.99998257e-01, 1.00000000e+00],
       ...,
       [4.66208127e-01, 4.22644047e-01, 4.79056858e-01, ...,
        6.07162926e-13, 2.21141538e-15, 1.47610044e-33],
       [4.66208127e-01, 5.48612952e-01, 5.90475162e-01, ...,
        4.28865921e-01, 8.81883972e-02, 8.40578627e-03],
       [4.66208127e-01, 5.34846092e-01, 5.28446808e-01, ...,
        9.87636997e-01, 9.99872762e-01, 9.99999996e-01]])

In [112]:
pricer = Pricer()
pricer.price(bs_strategy)

0.06404802539114345